In [ ]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

import sys
backtest_dir = 'C://backtest/backtest/'
if backtest_dir not in sys.path:
    sys.path.insert(0, backtest_dir)
    
from Backtest import *
from BayesianOptimization import *
from ADX_EMVStrategy import ADX_EMVStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [ ]:
def run_backtest(config, trading_data, ohlc_data, window_ADX = 10, window_EMV=40, n=10, m=10):
    window_ADX = int(window_ADX)
    window_EMV = int(window_EMV)
    n = int(n)
    m = int(m)
    config['title'] = "ADX_EMVStrategy" + "_" + str(window_ADX) + "_" + str(window_EMV) + "_" + str(n) + "_" + str(m)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")
    
    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = ADX_EMVStrategy(config, events_queue, data_handler,
                           window_ADX = window_ADX,
                           window_EMV=window_EMV, n=n, m=m)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    return (results['cum_returns'][-1] - 1)

## Setting

In [ ]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/ADX_EMVStrategy",
    "title": "ADX_EMVStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [ ]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [ ]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"
* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"
* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [ ]:
BO = BayesianOptimization(
    run_backtest,
    {'window_ADX': (1, 240),
    'window_EMV': (1, 120),
    'n': (1, 120),
    'm': (1, 120)},
    is_int = [1, 1, 1, 1], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=50, n_iter=100, acq="ei", xi=0.01, **gp_params)

In [ ]:
print(BO.res['max'])

In [ ]:
BO.maximize(init_points=0, n_iter=110, acq="ei", xi=0.0001, **gp_params)

In [ ]:
print(BO.res['max'])

In [ ]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

In [ ]:
f, ax = plt.subplots(2,2, figsize = (16, 16))
ax[0,0].scatter(BO.X[:,0].flatten(), BO.Y, c="red", s=10, zorder=10)
ax[0,0].set_title('$val$ $vs$ $short\_window$ ', fontsize=18)
ax[0,0].set_xlabel('$short\_window$', fontsize=16)
ax[0,0].set_ylabel('$val$', fontsize=16)
ax[0,0].hlines(0, 0, 120, color='black',linestyle="--")
ax[0,0].hlines(-1, 0, 120, color='black',linestyle="--")
ax[0,0].hlines(1, 0, 120, color='black',linestyle="--")
ax[0,0].vlines(BO.res['max']['max_params']['short_window'], -1.2, 2, color='blue',linestyle="--")

ax[0,1].scatter(BO.X[:,1].flatten() + BO.X[:,0].flatten(), BO.Y, c="red", s=10, zorder=10)
ax[0,1].set_title('$val$ $vs$ $long\_window$ ', fontsize=18)
ax[0,1].set_xlabel('$long\_window$', fontsize=16)
ax[0,1].set_ylabel('$val$', fontsize=16)
ax[0,1].hlines(0, 0, 240,color='black',linestyle="--")
ax[0,1].hlines(-1, 0, 240, color='black',linestyle="--")
ax[0,1].hlines(1, 0, 240,color='black',linestyle="--")
ax[0,1].vlines(BO.res['max']['max_params']['delta_window'] + BO.res['max']['max_params']['short_window'],
               -1.2, 2, color='blue',linestyle="--")

ax[1,0].scatter(BO.X[:,2].flatten(), BO.Y, c="red", s=10, zorder=10)
ax[1,0].set_title('$val$ $vs$ $window$ ', fontsize=18)
ax[1,0].set_xlabel('$window$', fontsize=16)
ax[1,0].set_ylabel('$val$', fontsize=16)
ax[1,0].hlines(0, 0, 240,color='black',linestyle="--")
ax[1,0].hlines(-1, 0, 240, color='black',linestyle="--")
ax[1,0].hlines(1, 0, 240,color='black',linestyle="--")
ax[1,0].vlines(BO.res['max']['max_params']['window'], -1.2, 2, color='blue',linestyle="--")

ax[1,1].scatter(BO.X[:,3].flatten(), BO.Y, c="red", s=10, zorder=10)
ax[1,1].set_title('$val$ $vs$ $a$ ', fontsize=18)
ax[1,1].set_xlabel('$a$', fontsize=16)
ax[1,1].set_ylabel('$val$', fontsize=16)
ax[1,1].hlines(0, 0, 3,color='black',linestyle="--")
ax[1,1].hlines(-1, 0, 3, color='black',linestyle="--")
ax[1,1].hlines(1, 0, 3,color='black',linestyle="--")
ax[1,1].vlines(BO.res['max']['max_params']['a'], -1.2, 2, color='blue',linestyle="--")

In [ ]:
## Acquisition Function "Probability of Improvement"

In [ ]:
* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)